<a href="https://colab.research.google.com/github/karanidenis/DQN/blob/main/q_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install stable-baselines3[extra] gym

# !pip install tensorflow
# !pip install keras
!pip install stable-baselines3[extra]
# import tensorflow as tf



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 8.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl.metadata (558 bytes)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.1

In [ ]:
import gym
from gym import spaces
from gym.envs.registration import register
import numpy as np
import pygame


class PatientRoutingEnv(gym.Env):
    metadata = {"render_modes": ["human", "rgb_array"], "render_fps": 1}

    def __init__(self, render_mode=None, size=5):
        super().__init__()
        self.size = size
        self.window_size = 800
        self.max_steps = 100

        self.observation_space = spaces.Box(
            low=0, high=size - 1, shape=(4,), dtype=int)
        self.action_space = spaces.Discrete(4)
        self._action_to_direction = {
            0: np.array([1, 0]),   # right
            1: np.array([0, 1]),   # up
            2: np.array([-1, 0]),  # left
            3: np.array([0, -1])   # down
        }

        assert render_mode is None or render_mode in self.metadata["render_modes"]
        self.render_mode = render_mode
        self.window = None
        self.clock = None

        self.obstacles = []
        self.goal = None
        self.current_reward = 0
        self.starting_point = np.array([0, 0])
        self.steps_taken = 0

    def _get_obs(self):
        return np.concatenate([self._agent_location, self._goal_location])

    def _get_info(self):
        return {"distance": np.linalg.norm(self._agent_location - self._goal_location, ord=1)}

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self._agent_location = self.starting_point.copy()
        self._goal_location = self.np_random.integers(
            0, self.size, size=2, dtype=int)
        while np.array_equal(self._goal_location, self._agent_location):
            self._goal_location = self.np_random.integers(
                0, self.size, size=2, dtype=int)

        self.obstacles = [
            (np.array([0, 2]), 'Nurse'),
            (np.array([3, 2]), 'Doctor'),
            (np.array([3, 4]), 'Restricted'),
            (np.array([4, 1]), 'Emergency'),
            (np.array([1, 4]), 'Ward'),
            (np.array([1, 4]), 'Doctor'),
        ]

        self.current_reward = 0
        self.steps_taken = 0

        observation = self._get_obs()
        info = self._get_info()

        if self.render_mode == "human":
            self._render_frame()

        return observation, info

    def step(self, action):
        direction = self._action_to_direction[action]
        new_location = np.clip(self._agent_location +
                               direction, 0, self.size - 1)

        reward = 0

        if new_location.tolist() not in [o[0].tolist() for o in self.obstacles]:
            self._agent_location = new_location
        else:
            self._agent_location = self.starting_point
            reward = -10

        terminated = np.array_equal(self._agent_location, self._goal_location)
        if terminated and self.steps_taken > 10:
            reward = 10
            self._agent_location = self._goal_location

        self.current_reward += reward
        self.steps_taken += 1

        done = terminated or self.steps_taken >= self.max_steps

        observation = self._get_obs()
        info = self._get_info()

        if self.render_mode == "human":
            self._render_frame()

        return observation, reward, done, False, info

    def render(self):
        if self.render_mode == "rgb_array":
            return self._render_frame()

    def _render_frame(self):
        if self.window is None and self.render_mode == "human":
            pygame.init()
            pygame.display.init()
            self.window = pygame.display.set_mode(
                (self.window_size, self.window_size))
        if self.clock is None and self.render_mode == "human":
            self.clock = pygame.time.Clock()

        canvas = pygame.Surface((self.window_size, self.window_size))
        canvas.fill((255, 255, 255))
        pix_square_size = self.window_size / self.size

        for obstacle, label in self.obstacles:
            pygame.draw.rect(
                canvas,
                (0, 0, 0),
                pygame.Rect(
                    pix_square_size * obstacle,
                    (pix_square_size, pix_square_size),
                ),
            )
            font = pygame.font.SysFont(None, 24)
            text = font.render(label, True, (255, 255, 255))
            canvas.blit(text, (pix_square_size *
                        obstacle[0] + 5, pix_square_size * obstacle[1] + 5))

        pygame.draw.rect(
            canvas,
            (255, 0, 0),
            pygame.Rect(
                pix_square_size * self._goal_location,
                (pix_square_size, pix_square_size),
            ),
        )
        font = pygame.font.SysFont(None, 24)
        text = font.render('ECG Room', True, (255, 255, 255))
        canvas.blit(text, (pix_square_size *
                    self._goal_location[0] + 5, pix_square_size * self._goal_location[1] + 5))

        pygame.draw.circle(
            canvas,
            (0, 0, 255),
            (self._agent_location + 0.5) * pix_square_size,
            pix_square_size / 3,
        )
        font = pygame.font.SysFont(None, 16)
        text = font.render('Patient', True, (255, 255, 255))
        canvas.blit(text, ((self._agent_location + 0.5) * pix_square_size - (pix_square_size / 6),
                    (self._agent_location + 0.5) * pix_square_size - (pix_square_size / 6)))

        for x in range(self.size + 1):
            pygame.draw.line(
                canvas,
                0,
                (0, pix_square_size * x),
                (self.window_size, pix_square_size * x),
                width=3,
            )
            pygame.draw.line(
                canvas,
                0,
                (pix_square_size * x, 0),
                (pix_square_size * x, self.window_size),
                width=3,
            )

        font = pygame.font.SysFont(None, 20)
        reward_text = font.render(
            f'Reward: {self.current_reward}', True, (0, 0, 0))
        canvas.blit(reward_text, (5, self.window_size - 25))

        if self.render_mode == "human":
            self.window.blit(canvas, canvas.get_rect())
            pygame.event.pump()
            pygame.display.update()
            self.clock.tick(self.metadata["render_fps"])
        else:
            return np.transpose(np.array(pygame.surfarray.pixels3d(canvas)), axes=(1, 0, 2))

    def close(self):
        if self.window is not None:
            pygame.display.quit()
            pygame.quit()


register(
    id='gym_examples/GridWorld-v0',
    entry_point='patient_routing_env:PatientRoutingEnv',
    max_episode_steps=1000,
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import gym
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv
# from patient_routing_env import PatientRoutingEnv
import numpy as np


def main():
    import tensorflow as tf
    print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

    # Initialize the environment with no rendering
    env = DummyVecEnv([lambda: PatientRoutingEnv(render_mode=None, size=5)])
    print("Environment initialized successfully.")

    # Define the model
    model = DQN(
        policy='MlpPolicy',
        env=env,
        verbose=1,
        learning_rate=1e-3,
        buffer_size=10000,
        learning_starts=1000,
        batch_size=64,
        tau=1.0,
        gamma=0.99,
        train_freq=4,
        target_update_interval=1000,
        exploration_fraction=0.2,
        exploration_final_eps=0.05,
        max_grad_norm=10,
    )

    # Training parameters
    total_episodes = 50  # Number of episodes to train
    total_timesteps_per_episode = 1000  # Increased timesteps per episode

    for episode in range(total_episodes):
        print(f"Starting training episode {episode + 1}...")
        model.learn(total_timesteps=total_timesteps_per_episode)
        print(f"Training episode {episode + 1} completed.")

    # Save the model
    model.save("dqn_patient_routing")

    # Close the environment
    env.close()


if __name__ == "__main__":
    main()

Num GPUs Available:  0
Environment initialized successfully.
Using cpu device


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Starting training episode 1...
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 5795     |
|    time_elapsed     | 0        |
|    total_timesteps  | 243      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 8        |
|    fps              | 6376     |
|    time_elapsed     | 0        |
|    total_timesteps  | 477      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 12       |
|    fps              | 6526     |
|    time_elapsed     | 0        |
|    total_timesteps  | 727      |
----------------------------------
------------------------